In [2]:
import numpy as np
import pandas as pd
import pymysql
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
import os
from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from math import sqrt
from sklearn import grid_search
from sklearn.ensemble import RandomForestClassifier

In [35]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)

In [3]:
db = pymysql.connect(host="rdsv313f3wf342zn75661o.mysql.rds.aliyuncs.com", port=3306, user='ningle', passwd='ningle@2018', db='erp_jst',
                     charset='utf8')
cursor = db.cursor()

In [7]:
sql = "SELECT shop_buyer_id,  TIMESTAMPDIFF(DAY, NOW(), MAX(order_date)) AS R, COUNT(shop_buyer_id) as F,SUM(pay_amount) AS M FROM \
       orders_2018 where not isnull(shop_buyer_id) and shop_buyer_id <> '' group by shop_buyer_id "
cursor.execute(sql)

data = cursor.fetchall()

In [11]:
columnlist = ['user_id', 'recency', 'frequency', 'monetary']
rfm = pd.DataFrame.from_records(list(data), columns=columnlist)

In [38]:
rfm_new = np.array(rfm[['frequency', 'monetary', 'recency']])
seed = 9

In [39]:
rfm_new1 = imp.fit_transform(rfm_new)

In [40]:
clf = KMeans(n_clusters=8, random_state=seed)

In [41]:
clf = clf.fit(rfm_new1)

In [42]:
clf.cluster_centers_

array([[ 1.43242923e+00,  9.47862663e+01, -3.01687072e+02],
       [ 8.00000000e+01,  5.14372957e+07, -1.34000000e+02],
       [ 1.00000000e+01,  1.40162905e+07, -2.83000000e+02],
       [ 7.60000000e+01,  6.91682807e+07, -1.29000000e+02],
       [ 3.83333333e+01,  4.25318750e+06, -2.09000000e+02],
       [ 1.49000000e+02,  4.29668182e+07, -1.29000000e+02],
       [ 3.75000000e+01,  6.67489286e+06, -1.27500000e+02],
       [ 7.24137931e+00,  7.84791678e+05, -2.91137931e+02]])

In [43]:
rfm['label'] = clf.labels_

In [47]:
c = rfm['label'].value_counts()

In [48]:
c

0    2576336
7         29
4          3
2          3
6          2
5          1
3          1
1          1
Name: label, dtype: int64